# Preliminaries

In [8]:
# import modules
import numpy as np
... # your code here

Ellipsis

In [23]:
# base classes

class Node:
    pass

class Tree:
    def __init__(self):
        self.root = Node()
    
    def find_leaf(self, x):
        node = self.root
        while hasattr(node, "feature"):
            j = node.feature
            if x[j] <= node.threshold:
                node = node.left
            else:
                node = node.right
        return node

In [22]:
# test 
a = np.array([[1,2,3], [4,5,6], [7,8,9]])
indices = a[:,0] > 2
print(np.prod(a, axis=0))
print(1/2/3)

[ 28  80 162]
0.16666666666666666


# Density Tree

In [17]:
class DensityTree(Tree):
    def __init__(self):
        super(DensityTree, self).__init__()
        
    def train(self, data, prior, n_min=20):
        '''
        data: the feature matrix for the digit under consideration
        prior: the prior probability of this digit
        n_min: termination criterion (don't split if a node contains fewer instances)
        '''
        self.prior = prior
        N, D = data.shape
        D_try = int(np.sqrt(D)) # number of features to consider for each split decision

        # find and remember the tree's bounding box, 
        # i.e. the lower and upper limits of the training feature set
        m, M = np.min(data, axis=0), np.max(data, axis=0)
        self.box = m.copy(), M.copy()
        
        # identify invalid features and adjust the bounding box
        # (If m[j] == M[j] for some j, the bounding box has zero volume, 
        #  causing divide-by-zero errors later on. We must exclude these
        #  features from splitting and adjust the bounding box limits 
        #  such that invalid features have no effect on the volume.)
        valid_features   = np.where(m != M)[0]
        invalid_features = np.where(m == M)[0]
        M[invalid_features] = m[invalid_features] + 1

        # initialize the root node
        self.root.data = data
        self.root.box = m.copy(), M.copy()

        # build the tree
        stack = [self.root]
        while len(stack):
            node = stack.pop()
            n = node.data.shape[0] # number of instances in present node
            if n >= n_min:
                # Call 'make_density_split_node()' with 'D_try' randomly selected 
                # indices from 'valid_features'. This turns 'node' into a split node
                # and returns the two children, which must be placed on the 'stack'.
                left, right = make_density_split_node(node, N, random.permutation(valid_features)[:D_try])
                stack.append(left)
                stack.append(right)
            else:
                # Call 'make_density_leaf_node()' to turn 'node' into a leaf node.
                make_density_leaf_node(node, N)

    def predict(self, x):
        leaf = self.find_leaf(x)
        # return p(x | y) * p(y) if x is within the tree's bounding box 
        # and return 0 otherwise
        if !hasattr(leaf, 'response'):
            return 0
        else:
            return self.prior * leaf.response

In [10]:
def make_density_split_node(node, N, feature_indices):
    '''
    node: the node to be split
    N:    the total number of training instances for the current class
    feature_indices: a numpy array of length 'D_try', containing the feature 
                     indices to be considered in the present split
    '''
    n, D = node.data.shape
    m, M = node.box

    # find best feature j (among 'feature_indices') and best threshold t for the split
    e_min = float("inf")
    j_min, t_min = None, None
    
    for j in feature_indices:
        # Hint: For each feature considered, first remove duplicate feature values using 
        # 'np.unique()'. Describe here why this is necessary.
        """if we don't use np.unique(), we will end up calculating the error of some identical thresholds for multiple times"""
        data_unique = np.sort(np.unique(node.data[:, j]))
        # Compute candidate thresholds
        tj = np.array([(data_unique[i]+data_unique[i+1])/2 for i in range(data_unique.shape-1)])
        
        # Illustration: for loop - hint: vectorized version is possible
        for t in tj:
            # Compute the error
            N_l = np.sum(node.data[:, j]<=t)
            N_r = np.sum(node.data[:, j]>t)
            V_l = np.prod(np.max(node.data[node.data[:, j]<=t], axis=0) - np.min(node.data[node.data[:, j]<=t], axis=0))
            V_r = np.prod(np.max(node.data[node.data[:, j]>t], axis=0) - np.min(node.data[node.data[:, j]>t], axis=0))
            loo_error = loo_error(N, N_l, V_l) + loo_error(N, N_r, V_r)
            
            # choose the best threshold that
            if loo_error < e_min:
                e_min = loo_error
                j_min = j
                t_min = t

    # create children
    left = Node()
    right = Node()
    
    # initialize 'left' and 'right' with the data subsets and bounding boxes
    # according to the optimal split found above
    left.data = node.data[node.data[:, j_min]<=t] # store data in left node -- for subsequent splits
    left.box = np.min(left.data, axis=0), np.max(left.data, axis=0) # store bounding box in left node
    right.data = node.data[node.data[:, j_min]>t]
    right.box = np.min(right.data, axis=0), np.max(right.data, axis=0)

    # turn the current 'node' into a split node
    # (store children and split condition)
    node.left = left
    node.right = right
    node.feature = j_min
    node.threshold = t_min

    # return the children (to be placed on the stack)
    return left, right

In [18]:
def make_density_leaf_node(node, N):
    '''
    node: the node to become a leaf
    N:    the total number of training instances for the current class
    '''
    # compute and store leaf response
    n = node.data.shape[0]
    v = np.prod(node.box[1]-node.box[0])
    node.response = n/(N*v)

In [ ]:
def loo_error(N, Nm, Vm):
    return (Nm/(N*Vm))*(Nm/N - 2*(Nm-1)/(N-1))

# Decision Tree

In [ ]:
class DecisionTree(Tree):
    def __init__(self):
        super(DecisionTree, self).__init__()
        
    def train(self, data, labels, n_min=20):
        '''
        data: the feature matrix for all digits
        labels: the corresponding ground-truth responses
        n_min: termination criterion (don't split if a node contains fewer instances)
        '''
        N, D = data.shape
        D_try = int(np.sqrt(D)) # how many features to consider for each split decision

        # initialize the root node
        self.root.data = data
        self.root.labels = labels
        
        stack = [self.root]
        while len(stack):
            node = stack.pop()
            n = node.data.shape[0] # number of instances in present node
            if n >= n_min and not node_is_pure(node):
                # Call 'make_decision_split_node()' with 'D_try' randomly selected 
                # feature indices. This turns 'node' into a split node
                # and returns the two children, which must be placed on the 'stack'.
                left, right = make_decision_split_node(node, np.random.permutaion(node.data.shape[1])[:D_try])
                stack.append(left)
                stack.append(right)
                ... # your code here
            else:
                # Call 'make_decision_leaf_node()' to turn 'node' into a leaf node.
                make_decision_leaf_node(node)
                ... # your code here
                
    def predict(self, x):
        leaf = self.find_leaf(x)
        # compute p(y | x)
        return leaf.response # your code here

In [22]:
def make_decision_split_node(node, feature_indices):
    '''
    node: the node to be split
    feature_indices: a numpy array of length 'D_try', containing the feature 
                     indices to be considered in the present split
    '''
    n, D = node.data.shape

    # find best feature j (among 'feature_indices') and best threshold t for the split
    e_min = float("inf")
    j_min, t_min = None, None
    
    for j in feature_indices:
        data_unique = np.sort(np.unique(node.data[:, j]))
        # Compute candidate thresholds
        tj = np.array([(data_unique[i]+data_unique[i+1])/2 for i in range(data_unique.shape-1)])
        
        # Illustration: for loop - hint: vectorized version is possible
        for t in tj:
            # Compute the error
            N_l = np.sum(node.data[:, j]<=t)
            N_r = np.sum(node.data[:, j]>t)
            V_l = np.prod(np.max(node.data[node.data[:, j]<=t], axis=0) - np.min(node.data[node.data[:, j]<=t], axis=0))
            V_r = np.prod(np.max(node.data[node.data[:, j]>t], axis=0) - np.min(node.data[node.data[:, j]>t], axis=0))
            loo_error = loo_error(N, N_l, V_l) + loo_error(N, N_r, V_r)
            
            # choose the best threshold that
            if loo_error < e_min:
                e_min = loo_error
                j_min = j
                t_min = t

    # create children
    left = Node()
    right = Node()
    
    # initialize 'left' and 'right' with the data subsets and labels
    # according to the optimal split found above
    left.data = ... # data in left node
    left.labels = ... # corresponding labels
    right.data = ...
    right.labels = ...

    # turn the current 'node' into a split node
    # (store children and split condition)
    node.left = left
    node.right = right
    node.feature = ...
    node.threshold = ...

    # return the children (to be placed on the stack)
    return left, right    

In [32]:
def make_decision_leaf_node(node):
    '''
    node: the node to become a leaf
    '''
    # compute and store leaf response
    node.N = ...
    node.response = ... # your code here

In [25]:
def node_is_pure(node):
    '''
    check if 'node' ontains only instances of the same digit
    '''
    return ... # your code here

# Evaluation of Density and Decision Tree

In [ ]:
# read and prepare the digits data
... # your code here

In [ ]:
# train trees, plot training error confusion matrices, and comment on your results
... # your code here

# Density and Decision Forest

In [7]:
class DensityForest():
    def __init__(self, n_trees):
        # create ensemble
        self.trees = [DensityTree() for i in range(n_trees)]
    
    def train(self, data, prior, n_min=20):
        for tree in self.trees:
            # train each tree, using a bootstrap sample of the data
            ... # your code here

    def predict(self, x):
        # compute the ensemble prediction
        return ... # your code here

In [35]:
class DecisionForest():
    def __init__(self, n_trees):
        # create ensemble
        self.trees = [DecisionTree() for i in range(n_trees)]
    
    def train(self, data, labels, n_min=0):
        for tree in self.trees:
            # train each tree, using a bootstrap sample of the data
            ... # your code here

    def predict(self, x):
        # compute the ensemble prediction
        return ... # your code here

# Evaluation of Density and Decision Forest

In [ ]:
# train forests (with 20 trees per forest), plot training error confusion matrices, and comment on your results
... # your code here